In [2]:
import pandas as pd
import numpy as np

# BMI가 추가된 데이터 불러오기 (경로 수정)
train_df = pd.read_csv('../preprocessed_data/train_processed_with_bmi.csv')
test_df = pd.read_csv('../preprocessed_data/test_processed_with_bmi.csv')

def create_bmi_category(bmi):
    """BMI 수치에 따라 카테고리를 분류하는 함수"""
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25:
        return 'Normal_BMI'
    elif 25 <= bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

def create_cholesterol_category(cholesterol):
    """콜레스테롤 수치에 따라 카테고리를 분류하는 함수"""
    if cholesterol < 200:
        return 'Normal_Cholesterol'
    elif 200 <= cholesterol < 240:
        return 'Borderline_High_Cholesterol'
    else:
        return 'High_Cholesterol'

def create_glucose_category(glucose):
    """혈당 수치에 따라 카테고리를 분류하는 함수"""
    if glucose < 100:
        return 'Normal_Glucose'
    elif 100 <= glucose < 126:
        return 'Pre-Diabetic'
    else:
        return 'Diabetic'

# BMI 카테고리 파생 변수 생성
train_df['BMI_category'] = np.vectorize(create_bmi_category)(train_df['BMI'])
test_df['BMI_category'] = np.vectorize(create_bmi_category)(test_df['BMI'])

# 콜레스테롤/혈당 카테고리 파생 변수 생성
train_df['cholesterol_category'] = np.vectorize(create_cholesterol_category)(train_df['cholesterol'])
test_df['cholesterol_category'] = np.vectorize(create_cholesterol_category)(test_df['cholesterol'])

train_df['glucose_category'] = np.vectorize(create_glucose_category)(train_df['glucose'])
test_df['glucose_category'] = np.vectorize(create_glucose_category)(test_df['glucose'])

# BMI 카테고리와 콜레스테롤/혈당 카테고리를 결합한 새로운 파생 변수 생성
train_df['health_status_combo'] = train_df['BMI_category'] + '_' + train_df['cholesterol_category'] + '_' + train_df['glucose_category']
test_df['health_status_combo'] = test_df['BMI_category'] + '_' + test_df['cholesterol_category'] + '_' + test_df['glucose_category']


# 원-핫 인코딩
categorical_cols = ['BMI_category', 'cholesterol_category', 'glucose_category', 'health_status_combo']
combined_df = pd.concat([train_df.drop('stress_score', axis=1), test_df], ignore_index=True)
combined_df = pd.get_dummies(combined_df, columns=categorical_cols, drop_first=True)

# 다시 train과 test 데이터 분리
train_processed_combo = combined_df.iloc[:len(train_df)].copy()
test_processed_combo = combined_df.iloc[len(train_df):].copy()
train_processed_combo['stress_score'] = train_df['stress_score']

# 새로운 파일로 저장
train_processed_combo.to_csv('../preprocessed_data/train_processed_combo.csv', index=False)
test_processed_combo.to_csv('../preprocessed_data/test_processed_combo.csv', index=False)

print("복합 파생 변수가 추가된 파일이 'preprocessed_data' 폴더에 저장되었습니다.")

복합 파생 변수가 추가된 파일이 'preprocessed_data' 폴더에 저장되었습니다.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 최종 전처리된 데이터 불러오기
train_df_combo = pd.read_csv('../preprocessed_data/train_processed_combo.csv')
test_df_combo = pd.read_csv('../preprocessed_data/test_processed_combo.csv')

# 피처(X)와 타겟(y) 분리
X_combo = train_df_combo.drop(['ID', 'stress_score'], axis=1)
y_combo = train_df_combo['stress_score']

# 훈련 데이터와 검증 데이터로 나누기
X_train_combo, X_val_combo, y_train_combo, y_val_combo = train_test_split(X_combo, y_combo, test_size=0.2, random_state=42)

# RandomForestRegressor 모델 초기화 및 학습
model_combo = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_combo.fit(X_train_combo, y_train_combo.values.ravel())

# 검증 데이터로 예측
y_pred_combo = model_combo.predict(X_val_combo)

# 모델 성능 평가 (RMSE)
rmse_combo = np.sqrt(mean_squared_error(y_val_combo, y_pred_combo))
print(f"복합 파생 변수 적용 모델의 검증 데이터 RMSE: {rmse_combo:.4f}")

# 테스트 데이터로 예측
final_predictions_combo = model_combo.predict(test_df_combo.drop('ID', axis=1))

# 제출 파일 생성
submission_df = pd.read_csv('../data/sample_submission.csv')
submission_df['stress_score'] = final_predictions_combo
submission_df.to_csv('submission_combo.csv', index=False)

print("복합 파생 변수가 적용된 모델의 제출 파일 'submission_combo.csv'가 성공적으로 생성되었습니다.")

복합 파생 변수 적용 모델의 검증 데이터 RMSE: 0.2353
복합 파생 변수가 적용된 모델의 제출 파일 'submission_combo.csv'가 성공적으로 생성되었습니다.
